# Support Vector Machine

A _support vector machine_ (SVM) is capable of performing linear or non-linear classification, regression and even outlier detection. SVMs are particularly useful for classification of complex but small or medium sized datasets.

### Linear SVM Classification
Consider the iris dataset, the two data classes can clearly be separated easily with a straight line (_linearly separable_). The plot shows a bad classifier (left) and a good classifier (right). The decision boundary of the left plot is bad because the dashed line cannot separate the classes properly and the solid lines are too close to the nearest instance of the two classes making it vulnerable to making errors on new instances. The right plot is a good classifier becuase the solid line is far from the nearest instance of the classes.

<img src="Figure%205-1.PNG" alt="Drawing" style="width: 600px;"/>

Imagine the plot as a street, adding training instances off the street will not affect the decision boundary, only the (highlighted) instances closest to the edge of the street will change the decision boundary. These instances are known as _support vectors_.

SVMs are sensitive to the feature scales. If the vertical scale is much larger than the horizontal scale, the widest possible street is close to the horizontal. After feature scaling, the decision boundary will be much better.

<img src="Figure%205-2.PNG" alt="Drawing" style="width: 600px;"/>

### Soft Margin Classification
If we impose that all instances be off the street and on the right side, this is known as _hard margin classification_. The problem with hard margin classification is that it only works if the data is linearly separable and also it is sensitive to outliers. Figure 5-3 shows the iris dataset with one additional outlier. A hard decision boundary cannot be found on the left plot while on the right plot the street is significantly narrower meaning the model will not generalise well.

<img src="Figure%205-3.PNG" alt="Drawing" style="width: 600px;"/>

The goal of soft margin classification is to use a more flexible model such that there is a balance where the streets are kept as large as possible while limiting _margin violations_ (instances that end up in the middle of the street or at the wrong side).

In Scikit-Learn's SVM classes, the balance can be controlled using the `C` hyperparameter: a small `C` value leads to a wider street but more margin violations. Figure 5-4 shows a non-linearly separable dataset. For a low `C` value (left), the margin is large but some instances end up on the street. For high `C` (right), the classifier makes few margin violations but has a smaller margin. The left classifier will generalise better and as seen most margin violations are actually on the correct side of the decision boundary. **If the SVM model is overfitting, trying regularising it by reducing `C`.**

<img src="Figure%205-4.PNG" alt="Drawing" style="width: 600px;"/>

To implement Scikit-Learn's SVM model (`LinearSVM` class with `C=1` and the _hinge loss_ function) to detect Iris-Virginica flowers, the iris dataset has to be loaded, feature scaled and trained. 

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) #Iris-Virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)

#Making a prediction
svm_clf.predict([[5.5, 1.7]])

array([1.])

Unlike Logistic Regression, SVM classifiers don't output probabilities for each class.

Alternatively, SVC class using `SVC(kernel="linear", C=1)` can also be used but is much slower, especially with large datasets. Another option is the `SGDClassifier` class with `SGDClassifier(loss="hinge", alpha=1/(m*C))`. This applies SGD to train a linear SVM classifier. It does not converge as fast as `LinearSVC` class but it can be useful to handle huge datasets that do not fit in memory (used in out-of-core training) or to handle online classification tasks.

The `LinearSVC` class regularises the bias term so the training set should first be centered by subtracting the mean. This is automatic if `StandardScaler` is used to scale the data. It is important to set the `loss` hyperparameter to `"hinge"`, as it is not the default value. For better performance, the `dual` hyperparameter should be set to `False` unless there are more features than training instances.

### Nonlinear SVM Classification
Although linear SVM classifiers work well, many datasets are not linearly separable. One approach to handling non-linear datasets is to add more features such as polynomial features; in some cases this can result in linearly separable dataset. Consider Figure 5-5 (left): it's a simple dataset with just one feature $x_1$. This dataset is not linearly separable. But if a second feature $x_2 = (x_1)^2$ is added, the 2D dataset is linearly separable.

<img src="Figure%205-5.PNG" alt="Drawing" style="width: 550px;"/>

To implement this in Scikit-Learn, a `Pipeline` class containing `PolynomialFeatures` transformer, followed by `StandardScaler` and a `LinearSVC`. Using the moon dataset (toy dataset for binary classification which datapoints are shaped as two interleaving half circles) to illustrate this idea: 

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

<img src="Figure%205-6.PNG" alt="Drawing" style="width: 420px;"/>

### Polynomial Kernel

Adding polynomial features is simple to implement and works well for many machine learning algorithms. However, a low polynomial degree cannot deal with very complex datasets but a high polynomial degree creates a huge number of features making the model slow.

Fortunately, using SVMs a technique called the _kernel trick_ can be used to get the same results as if many polynomial features were added, even with very high degree polynomials, without actually adding the polynomials. This means there is no combinatorial explosion of the number of features since no additional features are actually added. This is implemented in the `SVC` class using the moons dataset as an example:

In [4]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

The code trains the SVM using a 3<sup>rd</sup> degree polynomial kernel. On the right, it is a 10<sup>th</sup> degree polynomial kernel. If the model is overfitting, the polynomial degree should be reduced and vice versa. The hyperparameter `coef0` controls how much the model is influenced by the high degree polynomials vs. the low degree polynomials.
    
<img src="Figure%205-7.PNG" alt="Drawing" style="width: 550px;"/>

A common approach to find the right hyperparameter is to use grid search. It is faster to do a coarse grid search before using a fine grid search around the best values found. Knowing the range of hyperparameters helps finding the best values quicker.

### Adding Similarity Features
Another technique to tackle non-linear problems is to add features computed using a _similarity function_ that measures how much each instance resembles a particular _landmark_. For example, consider the one-dimensional dataset in Figure 5-5 and add two landmarks at $x_1 = -2$ and $x_1 = 1$. Define the similarity function to be the Gaussian _Radial Bias Function_ (RBF) with $y=0.3$

$$ \phi_y(\mathbf{x}, \ell) = \exp{(-\gamma \parallel\mathbf{x} - \ell\parallel ^2)} $$

It is a Bell Shaped Function varying from 0 (very far away from the landmark) to 1 (at the landmark). An example of how to compute the new features: take $x_1 = -1$: it is located at a distance of 1 from the first landmark, and 2 from the second landmark. Therefore, its new features are $ x_2 = \exp{-0.3\times1^2)} \approx 0.74 $ and $x_3 = \exp{(-0.3 \times 2^2)} \approx 0.30$. Figure 5-8 shows the transformed dataset (dropping the original features). The data after transformation is linearly separable.

<img src="Figure%205-8.PNG" alt="Drawing" style="width: 550px;"/>

To select landmarks, simplest approach is to create a landmark at the location of each and every instance in the dataset. This creates many dimensions and thus increases the chances that the transformed training set will be linearly separable. The problem is that a training set with $m$ instances and $n$ features gets transformed into a training set with $m$ instances and $m$ features (assuming the original features are dropped). If the training set is very large, then the number of features will be equally large.

### Gaussian RBF Kernel
Like polynomial features, similarity features method tends to be computationally expensive especially for large datasets. However, the kernel trick works making it possible to achieve similar results as if many similarity features were added without actually adding them. To implement Gaussian RBF kernel using the `SVC` class:

In [5]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

This example is plotted in the bottom left of Figure 5-9. The other plots show the model trained on different values of hyperparameters `gamma` and `C`. Increasing `gamma` makes the bell shaped curve narrower (left plot) and as a result each instance's range of influence is smaller: the decision boundary becomes more irregular, wiggling around individual instances. A small `gamma` value gives instances a larger range of influence and the decision boundary becomes smoother. $\gamma$ acts like a regularisation hyperparameter: **if the model is overfitting, reduce $\gamma$ and if it is underfitting then increase $\gamma$ (similar to `C` hyperparameter).**

<img src="Figure%205-9.PNG" alt="Drawing" style="width: 550px;"/>

Other kernels exist but are not as common. Some kernels are specialised for specific data structures. For example, _string kernels_ are sometimes used when classifying text documents or DNA sequences (e.g. using the _string subsequence kernel_ or kernels based on the _Levenshtein distance_).

To choose a kernel, it's recommended to try the linear kernel first (remember `LinearSVC` is much faster than `SVC(kernel="linear)`), especially if the training set is very large or if there are plenty of features. If the training set is not too large, then try Gaussian RBF kernel as well. Other kernels can be tried if there is time and computing power using cross validation and grid search, especially useful for kernels specialised for certain data structures.

### Computational Complexity
The `LinearSVC` class is based on the _liblinear_ library, which implements an __optimised algorithm__ for linear SVMs. It does not support the kernel trick, but it scales almost linearly with the number of training instances and the number of features: its training time complexity is roughly $O(m\times n)$.

The algorithm takes longer if you require a very high precision. This is controlled by the tolerance hyperparameter $\varepsilon$ (called `tol` in Scikit-Learn). In most classification tasks, the default tolerance is fine.

The `SVC` class is based on the _libsvm_ library, which implements an algorithm that supports the kernel trick. The training time complexity is usually between $O(m^2 \times n)$ and $O(m^3 \times n)$. Unfortunately, this means that when the number of training instances get large, computation becomes very slow. Therefore, it is better to use this algorithm on small to medium sized training sets. However, this method scales well with number of features, especially with _sparse features_ (i.e. when each instance has few nonzero features). In this case, the algorithm scales roughly with the average number of nonzero features per instance.

<img src="Table%205-1.PNG" alt="Drawing" style="width: 450px;"/>

### SVM Regression
SVM algorithm is quite versatile: it supports both linear and non-linear classification, but also linear and non-linear regression. The trick is to reverse the objective: instead of trying to fit the largest possible street between two classes while limiting margin violations, SVM Regression tries to fit as many instances as possible __on__ the street while limiting margin violations (i.e. instances __off__ the street). The width of the street is controlled by the hyperparameter $\varepsilon$. Figure 5-10 shows two linear SVM Regression models trained on some random linear data, one with a large margin ($\varepsilon = 1.5$) and other with a small margin ($\varepsilon=0.5$). 

<img src="Figure%205-10.PNG" alt="Drawing" style="width: 550px;"/>

Adding more training instances within the margin does not affect the model's prediction; thus, the model is said to be _$\varepsilon$-insensitive_.

Scikit-Learn's `LinearSVR` class can perform linear SVM Regression. The following code produces the model represented on the left of Figure 5-10 (training data should be scaled and centered first).

In [6]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

To tackle nonlinear regression tasks, a kernalised SVM model can be used. For example, Figure 5-11 shows SVM Regression on a random quadratic training set, using a 2<sup>nd</sup>-degree polynomial kernel. There is little regularisation on the left plot (i.e. a large `C` value) and much more regularisation on the right plot (i.e. a small `C` value).

<img src="Figure%205-11.PNG" alt="Drawing" style="width: 550px;"/>

The following code produces the model represented on the left Figure 5-11 using Scikit-Learn's `SVR` class which supports the kernel trick. The `SVR` class is the Regression equivalent of the `SVC` class. The `LinearSVR` class scales linearly with the size of the training set (just like `LinearSVC`), while `SVR` class gets really slow when handling large training sets. 

In [7]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Decision Function and Predictions
The linear SVM classifier model predicts the class of a new instance, $\mathbf{x}$, by simply computing the decision function $\mathbf{w}^T \mathbf{x} + b = w_1 x_1 + ... + w_n + x_n + b$: if the result is positive, the predicted class $\hat{y}$ is the positive class (1), or else it is the negative class (0)。

<img src="Equation%205-2.PNG" alt="Drawing" style="width: 350px;"/>

Figure 5-12 shows the decision function that corresponds to the model on the left of Figure 5-4; it is a two-dimensional plane since the dataset has two features (petal width and length). The decision boundary is the set of points where the decision function is **equal to zero**: it is the intersection of two planes which is a straight line (thick solid line).

<img src="Figure%205-12.PNG" alt="Drawing" style="width: 550px;"/>

The dashed lines represent the points where the decision function is equal to 1 or -1: they are parallel and equidistant to the decision boundary, forming a margin around it. Training a Linear SVM classifier means finding the value of $\mathbf{w}$ and $b$ that make this margin as wide as possible while avoiding margin violations or limiting them.

### Training Objective
Consider the slope of the decision function: it is equal to the norm of the weight vector, $\parallel \mathbf{w} \parallel$. If we divide the slope by 2, the points where the decision function is equal to $\pm1$ are going to be twice as far away from the decision boundary, i.e. dividing the slope by 2 will double the margin. The smaller the weight vector, $\parallel \mathbf{w} \parallel$, the larger the margin.

<img src="Figure%205-13.PNG" alt="Drawing" style="width: 550px;"/>

The objective is to minimise $\parallel \mathbf{w} \parallel$ to get a large margin. However, to avoid margin violation the decision function has to be greater than 1 for all positive traning instances and lower than -1 for all negative training instances. If we define $t^{(i)} = -1$ for negative instances (if $y^{(i)} = 0$) and $t^{(i)}=1$ for positive instances (if $y^{(i)}=1$, then this constraint can be expressed as $t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \geq 1$ for all instances.

The hard margin linear SVM classifier objective can be expressed as the _constrained optimization_ problem in Equation 5-3.

<img src="Equation%205-3.PNG" alt="Drawing" style="width: 350px;"/>

Note that we are minimising $\frac{1}{2} \mathbf{w}^T \mathbf{w}$, which is equal to $\frac{1}{2}\parallel\mathbf{w}\parallel^2$, rather than minimising $\parallel\mathbf{w}\parallel$. The derivative of $\frac{1}{2}\parallel\mathbf{w}\parallel^2$ is simply $\mathbf{w}$ but $\parallel\mathbf{w}\parallel$ is not differentiable at $\mathbf{w}=0$. Optimisation algorithms work best on differentiable functions.

To get the soft margin objective, we need to introduce a _slack variable_ $\zeta^{(i)} \geq 0$ for each instance: $\zeta^{(i)}$ measures how much the i<sup>th</sup> instance is allowed to violate the margin. There are two conflicting objectives: making the slack variables as small as possible to reduce margin violations and making $\frac{1}{2}\mathbf{w}^T\mathbf{w}$ as small as possible to increase the margin. This is what the `C` hyperparameter does: it allows us to define the trade-off between the two objectives. This results in Equation 5-4: a constrained optimisation problem

<img src="Equation%205-4.PNG" alt="Drawing" style="width: 450px;"/>

### Quadratic Programming
The hard and soft margin problems are both convex quadratic optimisation problems with linear constraints, also known as _Quadratic Programming_ (QP) problems. The general problem formulation is given by Equation 5-5.

<img src="Equation%205-5.PNG" alt="Drawing" style="width: 450px;"/>

Note that the expression $\mathbf{A} \mathbf{p} \leq \mathbf{b}$ actually defines $n_c$ constraints: $\mathbf{p}^T \mathbf{a}^{(i)} \leq b^{(i)} $ for $i=1, 2, ..., n_c$ where $\mathbf{a}^{(i)}$ is the vector containing the elements of the i<sup>th</sup> row of $\mathbf{A}$ and $b^{(i)}$ is the i<sup>th</sup> element of $\mathbf{b}$.

To verify QP parameters to get the hard margin linear SVM classifier objective:

- $n_p = n+1$, where n is the number of features (the +1 for the bias term)
- $n_c = n_m$, where m is the number of training instances
- $\mathbf{H}$ is the $n_p \times n_p$ identity matrix, except with a zero in the top left cell (to ignore the bias term)
- $\mathbf{f} = 0$, an $n_p$-dimensional vector full of 0s.
- $\mathbf{b} = -1$, an $n_c$-dimensional vector full of -1s.
- $\mathbf{a}^{(i)}=-t^{(i)} \dot{\mathbf{x}}^{(i)}$, where $\dot{\mathbf{x}}^{(i)}$ is equal to $\mathbf{x}^{(i)}$ with an extra bias feature $\dot{\mathbf{x}}_0 = 1$.

Passing in the QP parameters above to a off-the-shelf QP solver will result in a vector $\mathbf{p}$ which contains the bias term $b = p_0$ and the feature weights $w_i = p_i$ for $i=1, 2, ..., n$. Similarly, A QP solver can be used to solve the soft margin problem.

### The Dual Problem
Given a constrained optimisation problem, known as the _primal problem_, it is possible to express a different but closely related problem, called its _dual problem_. The solution of the dual problem typically gives a lower bound to the solution of the primal problem, but under some conditions it can have the same solutions as the primal problem. SVM problems meet these conditions, so solving the primal problem or the dual problem will yield the same results. Equation 5-6 shows the dual form of the linear SVM objective.

<img src="Equation%205-6.PNG" alt="Drawing" style="width: 400px;"/>

Once vector $\hat{\alpha}$ is found that minimises the equation (using a QP solver), then $\mathbf{w}$ and $\hat{b}$ can be computed that minimises the primal problem by using Equation 5-7.

<img src="Equation%205-7.PNG" alt="Drawing" style="width: 400px;"/>

The dual problem is faster to solve than the primal problem when the number of instances is smaller than the number of features. The dual problem also allows the kernel trick to be used whereas a primal problem does not.

### Kernalised SVM
Suppose you want to apply a 2<sup>nd</sup> degree polynomial transformation to a two-dimensional training set (like the moons training set), then train a linear SVM classifier on the transformed training set. Equation 5-8 shows the 2<sup>nd</sup> degree polynomial mapping function $\phi$ that is applied.

<img src="Equation%205-8.PNG" alt="Drawing" style="width: 400px;"/>

Notice that the transformed vector is three-dimensional instead of two-dimensional. Consider two two-dimensional vectors $\mathbf{a}$ and $\mathbf{b}$. If we apply this 2<sup>nd</sup> degree polynomial mapping then compute the dot product of the transformed vectors, it results in Equation 5-9.

<img src="Equation%205-9.PNG" alt="Drawing" style="width: 400px;"/>

The dot product of the transformed vectors is equal to the square of the dot product of the original vectors, i.e. $\phi(\mathbf{a})^T \phi(\mathbf{b}) = (\mathbf{a}^T \mathbf{b})^2$.

If we apply the transformation, $\phi$, to all training instances, then the dual problem (Equation 5-6) will contain the dot product $\phi(\mathbf{x}^{(i)})^T \phi(\mathbf{x}^{(j)})$. But if $\phi$ is the 2<sup>nd</sup>-degree polynomial transformation defined in Equation 5-8, then the dot product of the transformed vectors is simply $(\mathbf{x}^{(i)^T} \mathbf{x}^{(j)})^2$. The training instances don't need to be transformed and instead replace the dot product by its square in Equation 5-6. The result will be the same as transforming the training set then fitting a linear SVM algorithm.

The function $K(\mathbf{a}, \mathbf{b}) = (\mathbf{a}^T\mathbf{b})^2$ is called a 2<sup>nd</sup>-degree _polynomial kernel_. In ML, a _kernel_ is a function capable of computing the dot product $\phi(\mathbf{a})^T \phi(\mathbf{b})$ based only on the original vectors $\mathbf{a}$ and $\mathbf{b}$, without having to compute the transformation $\phi$.

### Mercer's Theorem
